# Small Data Variable Sellection by PLS

## Load dataset

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 1000)

In [2]:
#load sample data
df = pd.read_csv("train.csv")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,days,weeks
0,31,7,1,1,0,12294,1,0,0,21,11,101,3,498,0,1,0,326.0,47.0
1,29,2,2,2,0,43027,0,0,0,22,8,158,2,702,0,3,1,236.0,34.0
2,35,4,1,2,0,12252,1,0,0,11,11,351,1,826,0,0,0,316.0,45.0
3,31,9,1,1,0,99121,1,1,2,16,5,658,2,120,0,0,0,138.0,20.0
4,48,10,1,0,0,42005,1,0,1,3,4,177,1,273,0,3,0,94.0,13.0


In [23]:
df.shape

(27100, 19)

In [5]:
y = df["y"]
x = df.drop(["y"],axis=1)

In [6]:
# autoscaling
scaled_x = (x - x.mean(axis=0)) / x.std(axis=0, ddof=1)
scaled_y = (y - y.mean()) / y.std(ddof=1)

## Build PLS model

In [12]:
#https://nirpyresearch.com/partial-least-squares-regression-python/
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn import model_selection
import matplotlib.figure as figure

In [7]:
pls_component_number = 3
max_pls_component_number = 3 #set maximum component
fold_number = len(y) # Leave One Out
do_autoscaling = True  # True or False

In [14]:
#ref:https://datachemeng.com/wp-content/uploads/assignment14.py
pls_components = np.arange(1, min(np.linalg.matrix_rank(scaled_x) + 1, max_pls_component_number + 1), 1)
r2all = list()
r2cvall = list()
for pls_component in pls_components:
    pls_model_in_cv = PLSRegression(n_components=pls_component)
    pls_model_in_cv.fit(scaled_x, scaled_y)
    calculated_y_in_cv = np.ndarray.flatten(pls_model_in_cv.predict(scaled_x))
    estimated_y_in_cv = np.ndarray.flatten(
        model_selection.cross_val_predict(pls_model_in_cv, scaled_x, scaled_y, cv=fold_number))
    if do_autoscaling:
        calculated_y_in_cv = calculated_y_in_cv * y.std(ddof=1) + y.mean()
        estimated_y_in_cv = estimated_y_in_cv * y.std(ddof=1) + y.mean()

    r2all.append(float(1 - sum((y - calculated_y_in_cv) ** 2) / sum((y - y.mean()) ** 2)))
    r2cvall.append(float(1 - sum((y - estimated_y_in_cv) ** 2) / sum((y - y.mean()) ** 2)))
    #plt.plot(pls_components, r2all, 'bo-')
    #plt.plot(pls_components, r2cvall, 'ro-')
    #plt.ylim(0, 1)
    #plt.xlabel('Number of PLS components')
    #plt.ylabel('r2(blue), r2cv(red)')
    #plt.show()
    optimal_pls_component_number = np.where(r2cvall == np.max(r2cvall))
    optimal_pls_component_number = optimal_pls_component_number[0][0] + 1
    regression_model = PLSRegression(n_components=optimal_pls_component_number)

regression_model.fit(scaled_x, scaled_y)

PLSRegression(copy=True, max_iter=500, n_components=3, scale=True, tol=1e-06)

In [17]:
# calculate y
calculated_ytrain = np.ndarray.flatten(regression_model.predict(scaled_x))
if do_autoscaling:
    calculated_ytrain = calculated_ytrain * y.std(ddof=1) + y.mean()
# r2, RMSE, MAE
print('r2: {0}'.format(float(1 - sum((y - calculated_ytrain) ** 2) / sum((y - y.mean()) ** 2))))
print('RMSE: {0}'.format(float((sum((y - calculated_ytrain) ** 2) / len(y)) ** 0.5)))
print('MAE: {0}'.format(float(sum(abs(y - calculated_ytrain)) / len(y))))

r2: 0.09509939958509106
RMSE: 0.25500215631993856
MAE: 0.14005175281277182


## Calculate VIP & Beta

In [19]:
def calculate_vips(pls_model):
    t = pls_model.x_scores_
    w = pls_model.x_weights_
    q = pls_model.y_loadings_
    p, h = w.shape
    
    vips = np.zeros((p,))
    s = np.diag(np.matmul(np.matmul(np.matmul(t.T,t),q.T), q)).reshape(h, -1)
    total_s = np.sum(s)
    
    for i in range(p):
        weight = np.array([ (w[i,j] / np.linalg.norm(w[:,j]))**2 for j in range(h) ])
        vips[i] = np.sqrt(p*(np.matmul(s.T, weight))/total_s)
    return vips

In [20]:
vip = calculate_vips(regression_model)

In [27]:
vip[0]

1.0771203115365864

In [26]:
scaled_x.columns[0]

'age'

In [29]:
len(vip)

18

In [51]:
#Store variable and VIP
df2 = pd.DataFrame(columns=["Variables", "VIPs","Correlations"])

for i in range(len(vip)):
    va = scaled_x.columns[i]
    vi = vip[i]
    cor = y.corr(scaled_x.iloc[:,i])
    
    temp = pd.Series([va, vi, cor], index = df2.columns)
    df2 = df2.append(temp, ignore_index=True)

#In this model, input data is already standarization, so coeffients equal to Beta.
Beta = regression_model.coef_
df2["Beta"]=Beta
df2.head(20)

,Variables,VIPs,Correlations,Beta
0,age,1.077120,0.086668,0.065621
1,job,0.310702,-0.000511,-0.030827
2,marital,0.994208,0.081587,0.077347
3,education,0.926761,0.075080,0.054393
4,default,0.069551,-0.005586,-0.003757
5,balance,0.018802,-0.000899,-0.001056
6,housing,2.039587,-0.164710,-0.155568
7,loan,0.777421,-0.062696,-0.043159
8,contact,1.022646,-0.083513,-0.069899
9,day,0.254896,-0.011804,0.007804


## Ranking of Feature importance

In [68]:
#get abs value
df3=df2.copy()
df3["Correlations"]=abs(df3["Correlations"])
df3["Beta"]=abs(df3["Beta"])
df3.head()

,Variables,VIPs,Correlations,Beta
0,age,1.077120,0.086668,0.065621
1,job,0.310702,0.000511,0.030827
2,marital,0.994208,0.081587,0.077347
3,education,0.926761,0.075080,0.054393
4,default,0.069551,0.005586,0.003757


In [69]:
rank = df3.rank(ascending=False, numeric_only=True) #降順
rank=rank.rename(columns={"VIPs": "VIP_rank", "Correlations": "Corr_rank", "Beta":"Beta_rank"})
rank["Variables"]=df3["Variables"]
rank.head(20)

,VIP_rank,Corr_rank,Beta_rank,Variables
0,4.0,3.0,6.0,age
1,14.0,17.0,13.0,job
2,6.0,5.0,4.0,marital
3,7.0,6.0,7.0,education
4,16.0,15.0,16.0,default
5,17.0,16.0,18.0,balance
6,2.0,2.0,2.0,housing
7,10.0,7.0,8.0,loan
8,5.0,4.0,5.0,contact
9,15.0,13.0,15.0,day


In [70]:
rank["Total_Rank"]=np.sum(rank, axis=1)
rank.head(20)

,VIP_rank,Corr_rank,Beta_rank,Variables,Total_Rank
0,4.0,3.0,6.0,age,13.0
1,14.0,17.0,13.0,job,44.0
2,6.0,5.0,4.0,marital,15.0
3,7.0,6.0,7.0,education,20.0
4,16.0,15.0,16.0,default,47.0
5,17.0,16.0,18.0,balance,51.0
6,2.0,2.0,2.0,housing,6.0
7,10.0,7.0,8.0,loan,25.0
8,5.0,4.0,5.0,contact,14.0
9,15.0,13.0,15.0,day,43.0


In [72]:
rank_sort = rank.sort_values("Total_Rank")
rank_sort.head(10) #Get Top 10 important features

,VIP_rank,Corr_rank,Beta_rank,Variables,Total_Rank
14,1.0,1.0,1.0,previous,3.0
6,2.0,2.0,2.0,housing,6.0
0,4.0,3.0,6.0,age,13.0
8,5.0,4.0,5.0,contact,14.0
2,6.0,5.0,4.0,marital,15.0
15,3.0,14.0,3.0,poutcome,20.0
3,7.0,6.0,7.0,education,20.0
7,10.0,7.0,8.0,loan,25.0
16,8.0,8.0,11.0,days,27.0
17,9.0,9.0,12.0,weeks,30.0
